# Evaluation Model 7
## Model name : Yellow-AI-NLP/komodo-7b-base
## Link Colab : [Click](https://colab.research.google.com/drive/17c9iZs4p2Gs-V7aSDgOvhHN3vJjGMK-6?usp=sharing)
## Link Adapter weight : [Click](https://huggingface.co/farwew/Med-QA-komodo)

# Download Dependencies

In [ ]:
%%capture
! pip install unsloth
! pip install rouge_score
! pip install evaluate

In [ ]:
%%capture
! pip install -U peft
! pip install bert_score

# Import Dependencies

In [ ]:
from tqdm import tqdm
import kagglehub
import os

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import evaluate

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from datasets import load_dataset
import evaluate
import torch
import bert_score
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import os
import re

In [ ]:
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset

In [ ]:
from peft import PeftModel

# Load dataset

In [ ]:
# 1. Load dataset
dataset = load_dataset("farwew/DoctorsAnswerTextDataset-in-Indonesian")

README.md:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

(…)-00000-of-00001-e657306e4ba4079a.parquet:   0%|          | 0.00/17.9M [00:00<?, ?B/s]

(…)-00000-of-00001-6bdfe08bb1c69cf1.parquet:   0%|          | 0.00/948k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19845 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1048 [00:00<?, ? examples/s]

In [ ]:
# 3. Ambil data test
dataset_eval = dataset["test"]

# 4. Konversi ke pandas DataFrame
df = dataset_eval.to_pandas()
df.head()

,instruction,input,output,topic_set,topics
0,Selalu tersedak setiap makan dan minum,Selalu tersedak setiap makan dan minum. Dok sa...,batuk adalah respon tubuh untuk mengeluarkan b...,gastritis,penyakit-parkinson multiple-sclerosis gastritis
1,Apa obat untuk demam?,Apa obat untuk demam?. malam dok saya mau tany...,"kepala pusing, perut mual, punggung sakit dan ...",mual,flu pilek sakit-kepala nyeri-punggung mual
2,Cara mengatasi perut kembung pada anak,Cara mengatasi perut kembung pada anak. punya ...,perut kembung menandakan adanya penumpukan gas...,perut-kembung,anak gastritis perut-kembung
3,Perut Bagian Bawah Kembung,"Perut Bagian Bawah Kembung. saya mau bertanya,...","seperti: sayur: kol, brokoli, kembang kol, asp...",gangguan-pencernaan,gangguan-pencernaan
4,Nyeri dada menjalar ke punggung,Nyeri dada menjalar ke punggung. Malam dok tad...,saya coba bantu jawab. keluhan nyeri dada menj...,sakit-maag,nyeri-dada sakit-maag


In [ ]:
# 5. Stratified split berdasarkan kolom 'topic_set'
train_df, stratified_sample_df = train_test_split(
    df,
    test_size=100,  # ubah sesuai proporsi yang diinginkan
    stratify=df["topic_set"],
    random_state=42
)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
stratified_dataset = Dataset.from_pandas(stratified_sample_df)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

# Load Adapter

In [ ]:
# path = kagglehub.dataset_download("farhanwew/lora-adapter-1")
# print("Path to dataset files:", path)
# print(os.listdir(path + '/content'))
# full_path = path + '/content/lora_model'

#  Notebook Evaluasi Base Model (tanpa Adapter LoRA)

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Yellow-AI-NLP/komodo-7b-base", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = secret_value_0 # use one if using gated models like meta-llama/Llama-2-7b-hf
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Yellow-AI-NLP/komodo-7b-base does not have a padding token! Will use pad_token = <unk>.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(35008, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4

In [ ]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(35008, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4

In [ ]:
alpaca_prompt = (
    "Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan "
    "dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, "
    "sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n\n"
    "### Keluhan:\n{}\n\n"
    "### Detail Pertanyaan:\n{}\n\n"
    "### Jawaban:\n{}"
)

In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instr, inp, out in zip(instructions, inputs, outputs):
        prompt = chatml_medical_prompt_with_instruction.format(instr, inp, out) + tokenizer.eos_token
        texts.append(prompt)
    return {"text": texts}

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # <eos>

import re

def clean_medical_response(text: str) -> str:
    # Bersihkan dan kapitalisasi setiap kalimat
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    cleaned_sentences = []
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
        # Kapital awal
        sentence = sentence[0].upper() + sentence[1:] if len(sentence) > 1 else sentence.upper()
        # Tambahkan titik jika perlu
        if not sentence.endswith(('.', '!', '?')):
            sentence += '.'
        cleaned_sentences.append(sentence)
    return ' '.join(cleaned_sentences)

def formatting_prompts_func(examples):
    titles  = examples["instruction"]  # atau ganti dengan "title"
    inputs  = examples["input"]
    outputs = examples["output"]

    texts = []
    for title, input_text, output_text in zip(titles, inputs, outputs):
        cleaned_output = clean_medical_response(output_text)
        full_prompt = alpaca_prompt.format(title, input_text, cleaned_output) + EOS_TOKEN
        texts.append(full_prompt)

    return { "text": texts }

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Yellow-AI-NLP/komodo-7b-base",trust_remote_code=True, token = secret_value_0 )

A new version of the following files was downloaded from https://huggingface.co/Yellow-AI-NLP/komodo-7b-base:
- bahasallamatokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


## Inference dengan base model

In [ ]:
from transformers import TextStreamer

# Asumsikan tokenizer dan model sudah dimuat dan berada di CUDA
prompt = alpaca_prompt.format(
    "mencegah mual",
    "Anak saya mengalami mual, bagaimana mencegah mual?",
    ""
)

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        streamer=streamer  # ← streaming akan muncul di terminal langsung
    )

Mual dapat disebabkan oleh banyak hal, misalnya perut kosong, perut penuh, mengonsumsi makanan tertentu, menelan udara, dan lain-lain. Mual dapat disembuhkan dengan berbagai cara. Pertama, jika mual terjadi karena perut kosong, makanlah makanan ringan, misalnya biskuit, kue kering, atau roti. Kedua, jika mual terjadi karena perut penuh, makanlah makanan ringan, misalnya biskuit, kue kering, atau roti. Ketiga, jika mual terjadi karena mengonsumsi makanan tertentu, hindari mengonsumsi makanan tersebut. Keempat, jika mual terjadi karena menelan udara, jangan menelan udara. Kelima, jika mual terjadi karena terpapar bau yang kuat, hindari bau yang kuat. Keenam, jika mual terjadi karena kondisi tertentu, seperti saat naik pesawat, pusing, atau perut bergejala, gunakan penghambat mual yang dijual bebas di apotek.
### Sumber:
https://www.healthline.com/health/motion-sickness#treatment


## Evaluation

In [ ]:
# === Evaluasi === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_instructions = []
all_inputs = []

In [ ]:
def normalize_text(text):
    text = text.lower().strip()
    text = re.sub(r"\p{P}+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

In [ ]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(35008, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4

In [ ]:
# Cek device model
print(f"Model device: {next(model.parameters()).device}")

# Atau lebih detail
for name, param in model.named_parameters():
    print(f"{name}: {param.device}")
    break  # Cukup cek parameter pertama

Model device: cuda:0
model.embed_tokens.weight: cuda:0


In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Base Model"):
    # Membentuk prompt berdasarkan keluhan dan pertanyaan
    prompt = alpaca_prompt.format(
        example["instruction"],  # Keluhan
        example["input"],        # Detail pertanyaan
        ""                       # Jawaban dikosongkan untuk diisi model
    )

    # Tokenisasi prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate output dari model
    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=512,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            streamer=streamer
        )

    # Decode hasil output
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Ambil respons setelah "### Jawaban:"
    if "### Jawaban:" in decoded:
        response = decoded.split("### Jawaban:")[-1].strip()
    else:
        response = decoded.strip()  # fallback jika format berubah

    # Cetak beberapa contoh awal
    if num < 10:
        print("=" * 60)
        print("Prompt:\n", prompt)
        print("\nResponse:\n", response)
        print("=" * 60)

    # Simpan hasil evaluasi
    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append(example["instruction"])
    all_inputs.append(example["input"])

    num += 1

Evaluating Base Model:   0%|          | 0/100 [00:00<?, ?it/s]

diper Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Solusi atasi sakit maag yang tak kunjung sembuh

### Detail Pertanyaan:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan malam, a

Evaluating Base Model:   1%|          | 1/100 [00:13<22:53, 13.88s/it]

Prompt:
 Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Solusi atasi sakit maag yang tak kunjung sembuh

### Detail Pertanyaan:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan malam

Evaluating Base Model:   2%|▏         | 2/100 [00:41<35:46, 21.90s/it]

...

Keluhan lemas, pusing dan mual bisa disebabkan oleh banyak hal. Yang paling sering adalah karena kurang minum dan kurang makan.
Demam dan mual juga bisa disebabkan oleh berbagai hal.
Silakan konsultasi ke dokter umum atau dokter spesialis penyakit dalam untuk diperiksa lebih lanjut.
Diposkan oleh dr. Sara Wijaya, 09 April 2016
Mual dan pusing
Pusing, mual dan badan lemas
Pusing, mual dan badan lemas
Pusing dan badan lemas
Mual dan badan lemas
Mual, badan lemas dan pusing
Mual dan pusing
Mual dan badan lemas
Mual dan badan lemas
Mual, pusing dan badan lemas
Mual, badan lemas dan pusing
Mual, pusing dan badan lemas
Mual dan pusing
Mual, pusing dan badan lemas
Mual, badan lemas dan pusing
Mual dan badan lemas
Mual, pusing dan badan lemas
Mual dan badan lemas
Mual dan badan lemas
Mual, pusing dan badan lemas
Mual, badan lemas dan pusing
Mual, pusing dan badan lemas
Mual dan pusing
Mual, badan lemas dan pusing
Mual dan badan lemas
Mual dan pusing
Mual, pusing dan badan lemas
Mual dan badan 

Evaluating Base Model:  49%|████▉     | 49/100 [15:59<18:49, 22.14s/it]

Keluhan yang disampaikan dapat disebabkan oleh adanya infeksi pada saluran pencernaan yang ditandai dengan keluhan nyeri perut, sering sendawa dan sulit buang angin. Penyebab infeksi saluran pencernaan dapat disebabkan oleh bakteri atau virus.
Keluhan yang dialami suami Anda dapat disebabkan oleh infeksi bakteri pada saluran pencernaan.
Penyebab infeksi saluran pencernaan dapat disebabkan oleh bakteri atau virus.
Untuk mencegah kekambuhan Anda dapat melakukan langkah pencegahan seperti:
1. Makan makanan yang sehat dan bergizi
2. Hindari makanan yang merangsang perut
3. Cukup istirahat dan tidur
4. Hindari stres
5. Jika Anda perokok maka hentikan kebiasaan Anda tersebut
6. Hindari konsumsi alkohol
7. Hindari konsumsi obat sembarangan
8. Hindari konsumsi obat antibiotik tanpa resep dokter
9. Hindari konsumsi makanan yang pedas dan berminyak
10. Hindari konsumsi makanan yang berlemak
11. Hindari konsumsi minuman beralkohol
12. Hindari konsumsi kopi dan teh
13. Hindari konsumsi makanan yan

Evaluating Base Model:  50%|█████     | 50/100 [16:26<19:40, 23.62s/it]

Keluhan yang Anda alami kemungkinan merupakan keluhan akibat gangguan pada sistem saraf. Penyakit ini disebut dengan penyakit Parkinson. Penyakit Parkinson merupakan penyakit yang ditandai dengan adanya keluhan tremor (gemetaran) pada anggota gerak. Pada penyakit ini, keluhan tremor akan timbul pada saat tertentu, misalnya saat melakukan pekerjaan tertentu, saat berbicara atau saat 

In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/base_model_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI BASE MODEL SELESAI ===")
print(metrics)

In [ ]:
hasil_base = pd.read_csv('/kaggle/working/eval_basemodel_20250525_013945/base_model_outputs.csv')
hasil_base

In [ ]:
prnt()

# Dengan Adapter

In [ ]:
import torch
import gc

# Clear PyTorch cache
torch.cuda.empty_cache()

# Garbage collection
gc.collect()

In [ ]:
from google.colab import userdata
hg = userdata.get('HG')

In [ ]:
MAX_SEQ_LENGTH = 2048
USE_4BIT = True

# === LOAD LoRA MODEL === #
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "farwew/Med-QA-komodo-2",
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None,
    load_in_4bit = USE_4BIT,
    token = hg
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.39M [00:00<?, ?B/s]

Yellow-AI-NLP/komodo-7b-base does not have a padding token! Will use pad_token = <unk>.


adapter_model.safetensors:   0%|          | 0.00/640M [00:00<?, ?B/s]

Unsloth 2025.5.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(35008, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Line

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Yellow-AI-NLP/komodo-7b-base",trust_remote_code=True, token =hg )

tokenizer_config.json:   0%|          | 0.00/519k [00:00<?, ?B/s]

bahasallamatokenizer.py:   0%|          | 0.00/15.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Yellow-AI-NLP/komodo-7b-base:
- bahasallamatokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.39M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/58.1k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "mencegah mual",
    "Anak saya mengalami mual, bagaimana mencegah mual?",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s> Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
mencegah mual

### Detail Pertanyaan:
Anak saya mengalami mual, bagaimana mencegah mual?

### Jawaban:
mual merupakan suatu gejala yang dapat disebabkan oleh berbagai hal. mual dapat disebabkan oleh: gangguan pada saluran pencernaan, seperti gastroenteritis, keracunan makanan, radang usus, penyakit asam lambung, hepatitis, radang empedu, batu empedu, dan sebagainya gangguan pada sistem pernapasan, seperti pneumonia, tuberkulosis, bronkitis gangguan pada sistem saraf, seperti stroke, tumor otak, dan sebagainya gangguan pada sistem reproduksi, seperti kehamilan, kista ovarium, endometriosis, dan sebagainya mual yang dialami anak anda dapat disebabkan oleh salah satu penyebab di atas, atau dapat pula disebabkan oleh penyebab lain

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "Selalu tersedak setiap makan dan minum",
    "Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak...",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s> Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Selalu tersedak setiap makan dan minum

### Detail Pertanyaan:
Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak...

### Jawaban:
kondisi yang anda alami dengan keluhan utama berupa tersedak, dimana hal ini merupakan suatu kondisi dimana makanan atau minuman yang anda konsumsi masuk ke dalam saluran pernapasan, dapat disebabkan oleh berbagai macam hal. berikut ini beberapa hal yang dapat menyebabkan tersedak, diantaranya adalah : makan dan minum terlalu cepat, berbicara sambil makan, mengonsumsi makanan yang bertekstur keras, mengonsumsi makanan yang berlemak, mengonsumsi makanan yang 

In [ ]:
# === IMPORT TAMBAHAN === #
import bert_score
import pandas as pd
import json
from datetime import datetime
import os

# === LOAD METRIK EVALUASI === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_inputs = []  # Untuk menyimpan input
all_instructions = []  # Untuk menyimpan instruction

# === PARAMETER === #
batch_size = 1  # Sesuaikan dengan kapasitas GPU
MAX_DISPLAY = 10
displayed = 0

# === TIMESTAMP UNTUK NAMA FILE === #
timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"evaluation_results_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Fine tune Model"):
    prompt = alpaca_prompt.format(example["instruction"], example["input"], "")
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=1024,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    if num < 10:
        print(prompt)
        print()
        print('response:',response)

    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append(example["instruction"])
    all_inputs.append(example["input"])
    num+=1

Evaluating Fine tune Model:   1%|          | 1/100 [00:30<49:59, 30.30s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Solusi atasi sakit maag yang tak kunjung sembuh

### Detail Pertanyaan:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan malam, akibatn

Evaluating Fine tune Model:   2%|▏         | 2/100 [00:49<38:58, 23.86s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
solusi untuk mengatasi konstipasi

### Detail Pertanyaan:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
solusi untuk mengatasi konstipasi

### Detail Pertanyaan:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Jawaban:
kondisi ini dapat disebabkan oleh berbagai hal

Evaluating Fine tune Model:   3%|▎         | 3/100 [01:09<35:39, 22.06s/it]

...

Evaluating Fine tune Model:   8%|▊         | 8/100 [02:42<27:58, 18.25s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
sakit perut sebelah kiri bawah

### Detail Pertanyaan:
sakit perut sebelah kiri bawah. Dok, saya mau tanya. Saya sudah menikah dan berusia 21 tahun. Sudah hampir 2minggu lebih perut sebelah kiri bawah sakit. Terhitung dari selesai saya haid tanggal 30 maret 2016. Sakitnya sampai ke pinggang dan tanda tanda nya seperti akan haid. Karena pada saya pun terasa nyeri dan cenderung lebih sensitif. Di minggu ketiga ini sakitnya terasa ngilu sampai ke paha kiri. Bahkan kadang panggul panggul pun sakit. Itu kenapa ya dok?apakah saya hamil? Terima kasih

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sa

Evaluating Fine tune Model:   9%|▉         | 9/100 [03:08<31:16, 20.62s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Nyeri Ulu Hati Pasca Operasi Usus buntu

### Detail Pertanyaan:
Nyeri Ulu Hati Pasca Operasi Usus buntu. sudah 4 Bulan yang lalu saya di operasi usus buntu (Operasi Laparotomi) minggu minggu ini perut saya aga kurang enak dok. tadi pagi setelah BAB perut saya kram,melilit,perih....dan nyeri Ulu ati, nyerinya timbul tenggelam...tapi setelah minum obat maag (Omeprazol) sakitnya mereda tapi masih aga kurang enak perutnya itu penyebabnya apa ya dok? apa karena Maag atau ada iritasi di Usus? atau ada hubungannya dengan operasi usus buntu 4 bulan yang lalu?

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampu

Evaluating Fine tune Model: 100%|██████████| 100/100 [34:35<00:00, 20.76s/it]


In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/fine_tuned_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI Fine tune MODEL SELESAI ===")
print(metrics)

=== EVALUASI Fine tune MODEL SELESAI ===
{'bert_score': {'precision': 66.11, 'recall': 69.39, 'f1': 67.69}, 'bleu': 4.11, 'rouge': {'rouge1': np.float64(30.9), 'rouge2': np.float64(6.65), 'rougeL': np.float64(14.37)}}


In [ ]:
output_dir

'eval_basemodel_20250525_104038'

In [ ]:
! zip -r model_ft_8.zip output_dir eval_basemodel_20250525_104038

	zip warning: name not matched: output_dir
  adding: eval_basemodel_20250525_104038/ (stored 0%)
  adding: eval_basemodel_20250525_104038/fine_tuned_outputs.csv (deflated 77%)
  adding: eval_basemodel_20250525_104038/metrics.json (deflated 46%)


In [ ]:
!curl -F "reqtype=fileupload" -F "fileToUpload=@model_ft_8.zip" https://catbox.moe/user/api.php

https://files.catbox.moe/7gzeoa.zip